# **Gradient Check – Verificación Numérica de Gradientes**

Este notebook tiene como objetivo **verificar la correcta implementación del cálculo de gradientes** en la red neuronal del proyecto.  
El *Gradient Check* compara los gradientes obtenidos analíticamente (por backpropagation) con los calculados numéricamente mediante diferencias finitas.

---

### **1. Importación de módulos y configuración inicial**

En esta sección se importan los módulos necesarios desde la estructura del proyecto.  
También se ajusta el formato de impresión de `numpy` para una visualización más limpia.


In [ ]:
import os, sys
import numpy as np

sys.path.append(os.path.join(os.getcwd(), ".."))

from src.layers import Dense, ReLU, Tanh
from src.losses import CrossEntropyLoss, MSELoss
from src.network import NeuralNetwork

np.set_printoptions(precision=6, suppress=True)

### **2. Implementación de la función `gradient_check`**

La siguiente función realiza el *gradient checking* comparando:
-  **Gradiente analítico**: obtenido por backpropagation.
-  **Gradiente numérico**: calculado mediante diferencias centradas.

Si ambos gradientes son similares dentro de una tolerancia aceptable (`atol`, `rtol`),  
el test se considera exitoso.


In [11]:
def gradient_check(model, X, y, loss_fn, eps=1e-4, atol=1e-3, rtol=1e-2, verbose=True):
    logits = model.forward(X, training=True)
    base_loss = loss_fn.forward(logits, y)
    grad = loss_fn.backward(logits, y)
    model.backward(grad)

    analytic = model.grads()
    params = model.params()

    for name, W in params.items():
        g_ana = analytic[name]
        g_num = np.zeros_like(W)

        it = np.nditer(W, flags=["multi_index"], op_flags=["readwrite"])
        while not it.finished:
            idx = it.multi_index
            old = W[idx]

            W[idx] = old + eps
            l1 = loss_fn.forward(model.forward(X, training=True), y)

            W[idx] = old - eps
            l2 = loss_fn.forward(model.forward(X, training=True), y)

            W[idx] = old
            g_num[idx] = (l1 - l2) / (2 * eps)
            it.iternext()

        abs_diff = np.abs(g_num - g_ana)
        max_abs = abs_diff.max()
        denom = np.maximum(np.abs(g_num) + np.abs(g_ana), 1e-12)
        max_rel = (abs_diff / denom).max()

        print(f"[{name}] max|Δ|={max_abs:.3e}  max rel={max_rel:.3e}")
        assert (max_abs <= atol) or (max_rel <= rtol), f"Grad-check falló en {name}"

    print("Gradient check: TODOS los parámetros han pasado.")


### **3. Caso 1 – Clasificación con Cross Entropy (Red pequeña)**

En este primer caso probamos la red con una tarea de **clasificación multiclase**,  
utilizando la **CrossEntropyLoss**.  

El objetivo es verificar que los gradientes de las capas densas sean consistentes con los numéricos.


In [12]:
np.random.seed(0)
X = np.random.randn(6, 5).astype(np.float32)
y = np.array([0, 1, 2, 1, 0, 2], dtype=int)

model = NeuralNetwork([
    Dense(5, 7, init="xavier"), ReLU(),
    Dense(7, 3, init="xavier")
])

loss = CrossEntropyLoss()
gradient_check(model, X, y, loss, eps=1e-4, atol=2e-3, rtol=5e-2)

[W_0] max|Δ|=8.987e-04  max rel=2.026e-02
[b_0] max|Δ|=6.379e-04  max rel=7.430e-02
[W_2] max|Δ|=9.312e-04  max rel=2.829e-02
[b_2] max|Δ|=4.681e-04  max rel=1.835e-02
Gradient check: TODOS los parámetros han pasado.


##### **Conclusión del Caso 1 – Clasificación con Cross Entropy**

Los resultados del *gradient check* muestran una coincidencia muy alta entre los gradientes
analíticos obtenidos por backpropagation y los gradientes numéricos calculados mediante
diferencias finitas.

Los errores relativos máximos (`max rel`) se mantienen dentro de los umbrales definidos
(`rtol = 5e-2`), validando la correcta implementación de las derivadas en las capas densas
y en la función de pérdida **CrossEntropyLoss**.

En particular, los gradientes de las capas `Dense` y de los sesgos (`b`) presentan
desviaciones numéricas menores a las esperadas por precisión de coma flotante (float32).



### **4. Caso 2 – Regresión con MSE (Valores continuos)**

Aquí validamos el gradiente en una red neuronal que realiza **regresión** con salidas continuas,  
utilizando la **MSELoss (Mean Squared Error)**.

Esto permite comprobar que el gradiente funciona también en escenarios no categóricos.


In [13]:
np.random.seed(1)
X = np.random.randn(4, 3).astype(np.float32)
y = np.random.randn(4, 2).astype(np.float32)

model = NeuralNetwork([
    Dense(3, 6, init="xavier"), Tanh(),
    Dense(6, 2, init="xavier")
])

loss = MSELoss()
gradient_check(model, X, y, loss, eps=1e-4, atol=2e-3, rtol=5e-2)

[W_0] max|Δ|=3.688e-04  max rel=1.325e-02
[b_0] max|Δ|=4.436e-04  max rel=3.935e-03
[W_2] max|Δ|=3.195e-04  max rel=1.542e-03
[b_2] max|Δ|=2.540e-04  max rel=4.866e-03
Gradient check: TODOS los parámetros han pasado.


##### **Conclusión del Caso 2 – Regresión con MSE**

En este segundo experimento se evaluó el cálculo de gradientes en una red neuronal de regresión
utilizando la función de pérdida **MSELoss (Mean Squared Error)**.

Los valores obtenidos de `max|Δ|` y `max rel` se mantienen en el orden de **1e-3 a 1e-2**, lo cual
confirma que los gradientes analíticos y numéricos son equivalentes dentro del margen de
error aceptable para cálculos en precisión simple.

Este resultado demuestra que el flujo de gradientes funciona correctamente tanto para
tareas de clasificación como para regresión, y que la propagación de errores hacia atrás
se implementó de manera consistente en toda la arquitectura.